In [1]:
%load_ext autoreload
%load_ext autotime

time: 369 µs (started: 2023-07-16 20:05:57 -07:00)


In [2]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')

time: 645 µs (started: 2023-07-16 20:05:57 -07:00)


In [3]:
import dask
import dask.dataframe as dd
import json
import numpy as np
import pandas as pd
from dask import compute, delayed
from dask.distributed import Client, Future, LocalCluster, as_completed, get_client, wait
from typing import Union

time: 5.57 s (started: 2023-07-16 20:05:58 -07:00)


In [4]:
client = Client(LocalCluster(n_workers=16, local_directory=f"/var/tmp/playground-cm1-challenges"))
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 48,Total memory: 125.47 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37353,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 48
Started: Just now,Total memory: 125.47 GiB
Comm: tcp://127.0.0.1:41179,Total threads: 3
Dashboard: http://127.0.0.1:38321/status,Memory: 7.84 GiB
Nanny: tcp://127.0.0.1:41051,


time: 3.44 s (started: 2023-07-16 20:06:03 -07:00)


In [5]:
# log_dir = "/p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet"
log_dir = "/p/vast1/iopp/recorder_app_logs/montage_pegasus/nodes-32/_parquet"

time: 722 µs (started: 2023-07-16 20:06:07 -07:00)


In [6]:
all_view = dd.read_parquet(f"{log_dir}/*.parquet")
all_view

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
npartitions=36,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,object,int64,int32,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 315 ms (started: 2023-07-16 20:06:07 -07:00)


In [7]:
all_view.index.count().compute()

12346353

time: 11.1 s (started: 2023-07-12 13:54:03 -07:00)


In [10]:
all_view['file_dir'] = all_view['file_name'].apply(lambda x: '/'.join(x.split('/')[:-1]))
all_view.compute()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,...,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id,file_dir
0,1,132672,0,321552,0,3,2.137661,2.137760,fopen64,0,...,mProject#lassen192#132672#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,0,0,0.000000,0.000099,21377103,4007557435994691998,2303625805466626068,/p/gpfs1/haridev/iopp/montage-workflow/scratch...
1,2,132672,0,321552,0,1,2.137835,2.138117,fread,0,...,mProject#lassen192#132672#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,2,0,0.006764,0.000282,21379759,4007557435994691998,2303625805466626068,/p/gpfs1/haridev/iopp/montage-workflow/scratch...
2,3,132672,0,321552,0,3,2.138125,2.138141,fclose,0,...,mProject#lassen192#132672#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,0,0,0.000000,0.000016,21381327,4007557435994691998,2303625805466626068,/p/gpfs1/haridev/iopp/montage-workflow/scratch...
3,4,132672,0,321552,0,3,2.138159,2.138171,fopen64,0,...,mProject#lassen192#132672#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,0,0,0.000000,0.000012,21381647,4007557435994691998,2303625805466626068,/p/gpfs1/haridev/iopp/montage-workflow/scratch...
4,5,132672,0,321552,0,1,2.138231,2.138246,fread,0,...,mProject#lassen192#132672#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,2,0,0.127157,0.000015,21382382,4007557435994691998,2303625805466626068,/p/gpfs1/haridev/iopp/montage-workflow/scratch...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286057,2383210,63830,0,321552,0,2,18.125608,18.125610,fwrite,0,...,mAdd#lassen734#63830#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,2880,0,2746.582031,0.000001,181256095,4007557436607364090,369428171584890900,/p/gpfs1/haridev/iopp/montage-workflow/scratch...
286058,2383211,63830,0,321552,0,3,18.125628,18.126268,fclose,0,...,mAdd#lassen734#63830#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,0,0,0.000000,0.000640,181259478,4007557436607364090,369428171584890900,/p/gpfs1/haridev/iopp/montage-workflow/scratch...
286059,2383212,63830,0,321552,0,1,18.126276,18.131586,fread,0,...,mAdd#lassen734#63830#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,2880,0,0.517237,0.005310,181289317,4007557438554618451,369428171584890900,/p/gpfs1/haridev/iopp/montage-workflow/scratch...
286060,2383213,63830,0,321552,0,2,18.131601,18.131601,fwrite,0,...,mAdd#lassen734#63830#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,2880,0,0.000000,0.000000,181316017,4007557438554618451,369428171584890900,/p/gpfs1/haridev/iopp/montage-workflow/scratch...


time: 1min 18s (started: 2023-06-26 12:15:47 -07:00)


## Eval I: Data Reduction

In [13]:
all_view.dtypes

index          int64
proc           int64
rank           int32
thread_id      int32
cat            int32
io_cat         int32
tstart       float32
tend         float32
func_id       object
level          int32
hostname      object
app           object
proc_name     object
file_name     object
size           int64
acc_pat        int32
bandwidth    float32
duration     float32
tmid           int64
file_id        int64
proc_id        int64
dtype: object

time: 7.52 ms (started: 2023-07-15 13:20:31 -07:00)


In [31]:
all_view.compute().infer_objects().dtypes

index          int64
proc           int64
rank           int32
thread_id      int32
cat            int32
io_cat         int32
tstart       float32
tend         float32
func_id       object
level          int32
hostname      object
app           object
proc_name     object
file_name     object
size           int64
acc_pat        int32
bandwidth    float32
duration     float32
tmid           int64
file_id        int64
proc_id        int64
dtype: object

time: 29 s (started: 2023-07-15 13:27:22 -07:00)


In [28]:
acc_pat = all_view['acc_pat']
acc_pat_mem = acc_pat.memory_usage(deep=True).compute()
acc_pat2 = acc_pat.astype(np.bool_)
acc_pat2_mem = acc_pat2.memory_usage(deep=True).compute()
print('acc_pat dtype improvement (times):', acc_pat_mem/acc_pat2_mem)

acc_pat dtype improvement (times): 3.9988807348675137
time: 3.21 s (started: 2023-07-15 13:25:02 -07:00)


In [25]:
acc_pat.value_counts().compute()

0    12345330
1        1023
Name: acc_pat, dtype: int64

time: 1.43 s (started: 2023-07-15 13:24:41 -07:00)


In [26]:
acc_pat2.value_counts().compute()

False    12345330
True         1023
Name: acc_pat, dtype: int64

time: 1.39 s (started: 2023-07-15 13:24:48 -07:00)


In [22]:
io_cat = all_view['io_cat']
io_cat_mem = io_cat.memory_usage(deep=True).compute()
io_cat2 = io_cat.astype(np.int8)
io_cat2_mem = io_cat2.memory_usage(deep=True).compute()
print('io_cat dtype improvement (times):', io_cat_mem/io_cat2_mem)

io_cat dtype improvement (times): 3.9988807348675137
time: 2.39 s (started: 2023-07-15 13:24:02 -07:00)


In [23]:
io_cat.value_counts().compute()

1    6683714
2    5454853
3     207786
Name: io_cat, dtype: int64

time: 1.38 s (started: 2023-07-15 13:24:17 -07:00)


In [24]:
io_cat2.value_counts().compute()

1    6683714
2    5454853
3     207786
Name: io_cat, dtype: int64

time: 1.15 s (started: 2023-07-15 13:24:25 -07:00)


**File Name vs File ID**

In [7]:
file_name = all_view['file_name']
file_name_mem = file_name.memory_usage(deep=True).compute()
file_id = all_view['file_id']
file_id_mem = file_id.memory_usage(deep=True).compute()
print('file_id dtype improvement (times):', file_name_mem/file_id_mem)

file_id dtype improvement (times): 17.631651907126056
time: 8.1 s (started: 2023-07-16 20:06:16 -07:00)


In [17]:
file_id.unique().count().compute()

19680

time: 1.81 s (started: 2023-07-16 20:16:43 -07:00)


**Process Name vs Process ID**

In [35]:
proc_name = all_view['proc_name']
proc_name_mem = proc_name.memory_usage(deep=True).compute()
proc_id = all_view['proc_id']
proc_id_mem = proc_id.memory_usage(deep=True).compute()
print('proc_id dtype improvement (times):', proc_name_mem/proc_id_mem)

proc_id dtype improvement (times): 11.88138911910808
time: 3.36 s (started: 2023-07-15 13:32:43 -07:00)


In [37]:
proc_id.unique().count().compute()

11488

time: 1.32 s (started: 2023-07-15 14:07:29 -07:00)


**Func ID vs Categorical Func ID**

In [13]:
func_id = all_view['func_id']
func_id_mem = func_id.memory_usage(deep=True).compute()
func_id2 = all_view['func_id'].astype('category')
func_id2_mem = func_id2.memory_usage(deep=True).compute()
print('func_id dtype improvement (times):', func_id_mem/func_id2_mem)

func_id dtype improvement (times): 62.30799010444749
time: 10.3 s (started: 2023-07-16 13:47:54 -07:00)


In [14]:
func_id.value_counts().compute()

fread        6683714
fwrite       5454853
fopen64        93434
fclose         81598
vfprintf       11572
pipe           11518
__xstat64       6782
remove          1518
fseek           1361
unlink             3
Name: func_id, dtype: int64

time: 5.79 s (started: 2023-07-16 14:23:07 -07:00)


In [11]:
file_name_computed = file_name.compute()
file_id_compute = file_id.compute()

time: 4.92 s (started: 2023-07-16 20:09:11 -07:00)


In [15]:
%%timeit -r 20 -n 1

_ = file_name_computed.sort_values()

11.7 s ± 263 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)
time: 3min 53s (started: 2023-07-16 20:10:43 -07:00)


In [16]:
%%timeit -r 20 -n 1

_ = file_id_compute.sort_values()

2.47 s ± 177 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)
time: 49.6 s (started: 2023-07-16 20:14:37 -07:00)


## Eval II: Preprocessing - Query via string vs. integer indices

In [7]:
file_grp = all_view.groupby(['file_id', 'file_name']).agg({'size': sum})
files = file_grp.index.compute()
files

MultiIndex([(-9222633358449720911,          '3-diff.000017.000048.fits'),
            (-9221343239341138544,            '2-fit.000005.000017.txt'),
            (-9220642067073431512,            '2-fit.000020.000022.txt'),
            (-9219592665727134543,            '2-fit.000020.000032.txt'),
            (-9218362651926525491,            '2-fit.000015.000047.txt'),
            (-9216754904262680462,          '3-diff.000026.000039.fits'),
            (-9216382566996869448,          '1-diff.000048.000057.fits'),
            (-9215461525784070457,            '3-fit.000043.000057.txt'),
            (-9214978144544274239,     '1-diff.000017.000026_area.fits'),
            (-9214444624381405612,            '1-fit.000043.000047.txt'),
            ...
            ( 8857004603826425559,       'cposs2ukstu_red_001_004.fits'),
            ( 8927141822935412591,  'cposs2ukstu_red_007_002_area.fits'),
            ( 8948728582963324486,  'cposs2ukstu_red_006_007_area.fits'),
            ( 89953906

time: 8.99 s (started: 2023-07-03 19:54:34 -07:00)


In [8]:
file_ids = files.get_level_values(0)
file_names = files.get_level_values(1)

time: 4.73 ms (started: 2023-07-03 19:54:54 -07:00)


In [9]:
file_id_view = all_view.set_index('file_id').compute()
file_name_view = all_view.set_index('file_name').compute()

time: 45.1 s (started: 2023-07-03 19:54:56 -07:00)


In [10]:
(19680 - (19680 % 2 ** 3)) / 2 ** 3

2460.0

time: 7.08 ms (started: 2023-07-03 19:55:47 -07:00)


In [11]:
from time import time


def m(l, nel):
    return l[:int(nel)].memory_usage(deep=True)


def x(df, l, nel):
    # print(l, nel)
    t0 = time()
    df.loc[l[0]: l[int(nel) - 1]]
    t1 = time() - t0
    # print(f"time,{t1}")
    memory_usage = m(l, nel)
    # print(f"mem,{memory_usage}")

time: 1.69 ms (started: 2023-07-03 19:55:52 -07:00)


In [12]:
mnel = (19680 - (19680 % 2 ** 3)) / 2 ** 3
nels = [mnel, mnel * 2, mnel * 4]
nels

[2460.0, 4920.0, 9840.0]

time: 8.18 ms (started: 2023-07-03 19:55:56 -07:00)


### Integer range queries

In [13]:
%%timeit -r 10 -n 1000

x(file_id_view, file_ids, nels[0])

157 µs ± 40 µs per loop (mean ± std. dev. of 10 runs, 1000 loops each)
time: 1.57 s (started: 2023-07-03 19:56:08 -07:00)


In [14]:
m(file_ids, nels[0])

19680

time: 5.47 ms (started: 2023-07-03 19:56:11 -07:00)


In [15]:
%%timeit -r 10 -n 1000

x(file_id_view, file_ids, nels[1])

187 µs ± 15.6 µs per loop (mean ± std. dev. of 10 runs, 1000 loops each)
time: 1.88 s (started: 2023-07-03 19:56:12 -07:00)


In [16]:
m(file_ids, nels[1])

39360

time: 5.98 ms (started: 2023-07-03 19:56:14 -07:00)


In [17]:
%%timeit -r 10 -n 1000

x(file_id_view, file_ids, nels[2])

147 µs ± 8.19 µs per loop (mean ± std. dev. of 10 runs, 1000 loops each)
time: 1.47 s (started: 2023-07-03 19:56:15 -07:00)


In [18]:
m(file_ids, nels[2])

78720

time: 4.08 ms (started: 2023-07-03 19:56:16 -07:00)


### String range queries

In [40]:
%%timeit -r 10 -n 1000

x(file_name_view, file_names, nels[0])

644 µs ± 7.56 µs per loop (mean ± std. dev. of 10 runs, 1000 loops each)
time: 6.45 s (started: 2023-07-03 20:13:29 -07:00)


In [41]:
m(file_names, nels[0])

200270

time: 3.91 ms (started: 2023-07-03 20:13:36 -07:00)


In [42]:
%%timeit -r 10 -n 1000

x(file_name_view, file_names, nels[1])

994 µs ± 11.5 µs per loop (mean ± std. dev. of 10 runs, 1000 loops each)
time: 9.94 s (started: 2023-07-03 20:13:36 -07:00)


In [43]:
m(file_names, nels[1])

400612

time: 4.18 ms (started: 2023-07-03 20:13:46 -07:00)


In [44]:
%%timeit -r 10 -n 1000

x(file_name_view, file_names, nels[2])

1.7 ms ± 27.3 µs per loop (mean ± std. dev. of 10 runs, 1000 loops each)
time: 17 s (started: 2023-07-03 20:13:46 -07:00)


In [45]:
m(file_names, nels[2])

895545

time: 5.55 ms (started: 2023-07-03 20:14:03 -07:00)


In [46]:
%%timeit -n 1 -r 5

for nel in nels:
    x(file_id_view, file_ids, nel)

766 µs ± 336 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)
time: 6.09 ms (started: 2023-07-03 20:14:03 -07:00)


In [26]:
%%timeit -n 1 -r 5

for nel in nels:
    x(file_name_view, file_names, nel)

3.92 ms ± 184 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)
time: 21.7 ms (started: 2023-07-03 19:56:50 -07:00)


## Eval III: Preprocessing - Iteration vs. preprocessing

In [34]:
all_view['file_dir'] = all_view['file_name'].apply(lambda x: '/'.join(x.split('/')[:-1]))
all_view.groupby(['file_dir']).agg({'size': sum}).sort_values('size').compute()

,size
file_dir,
,0
/p/gpfs1/iopp/temp/montage,0
/p/gpfs1/iopp/temp/montage/2854964,0
/p/gpfs1/iopp/temp/montage/2854964/30,24072977
/p/gpfs1/iopp/temp/montage/2854964/7,72092250
...,...
/p/gpfs1/iopp/temp/montage/2854964/6/run,1421042712
/p/gpfs1/iopp/temp/montage/2854964/0/run,3254544638
/p/gpfs1/iopp/temp/montage/2854964/25/run,4237098833


time: 15 s (started: 2023-06-21 15:29:23 -07:00)


In [37]:
all_view['file_dir_id'] = all_view['file_id'] & 0b0111111111111111111111111111111100000000000000000000000000000000
all_view.compute()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,...,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id,file_dir,file_dir_id
0,1,111084,0,314672,0,3,0.242405,0.242414,__xstat64,0,...,/p/gpfs1/iopp/temp/montage/2854964/2,0,0,0.000000,0.000009,2424094,1571755611461267775,-4245390515581891276,/p/gpfs1/iopp/temp/montage/2854964,1571755608527339520
1,2,111084,0,314672,0,3,0.242502,0.242732,fopen64,0,...,/p/gpfs1/iopp/temp/montage/2854964/2/run/Kimag...,0,0,0.000000,0.000230,2426168,7638324191939094055,-4245390515581891276,/p/gpfs1/iopp/temp/montage/2854964/2/run,7638324191741083648
2,3,111084,0,314672,0,3,0.242851,0.242868,opendir,0,...,/p/gpfs1/iopp/temp/montage/2854964/2,0,0,0.000000,0.000017,2428593,1571755611461267775,-4245390515581891276,/p/gpfs1/iopp/temp/montage/2854964,1571755608527339520
3,4,111084,0,314672,0,3,0.242946,0.242956,__xstat64,0,...,/p/gpfs1/iopp/temp/montage/2854964/2,0,0,0.000000,0.000010,2429507,1571755611461267775,-4245390515581891276,/p/gpfs1/iopp/temp/montage/2854964,1571755608527339520
4,5,111084,0,314672,0,3,0.243006,0.243016,__xstat64,0,...,/p/gpfs1/iopp/temp/montage/2854964,0,0,0.000000,0.000010,2430107,5347462497940582382,-4245390515581891276,/p/gpfs1/iopp/temp/montage,5347462496607272960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235304,2332457,41731,31,317888,0,2,1.604730,1.604747,fwrite,0,...,/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc33...,96920,0,5469.237793,0.000017,16047384,7686174940287124680,8238577613900339652,/p/gpfs1/iopp/temp/montage/2854964/1/run,7686174937781895168
235305,2332458,41731,31,317888,0,2,1.604971,1.604992,fwrite,0,...,/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc33...,96920,0,4359.911133,0.000021,16049814,7686174937900816169,8238577613900339652,/p/gpfs1/iopp/temp/montage/2854964/1/run,7686174937781895168
235306,2332459,41731,31,317888,0,3,1.605062,1.605241,fclose,0,...,/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc33...,0,0,0.000000,0.000179,16051513,7686174940287124680,8238577613900339652,/p/gpfs1/iopp/temp/montage/2854964/1/run,7686174937781895168
235307,2332460,41731,31,317888,0,3,1.605244,1.605379,fclose,0,...,/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc33...,0,0,0.000000,0.000135,16053112,7686174937900816169,8238577613900339652,/p/gpfs1/iopp/temp/montage/2854964/1/run,7686174937781895168


time: 54 s (started: 2023-06-21 15:33:21 -07:00)


In [35]:
all_view['file_dir_id'] = all_view['file_id'] & 0b0111111111111111111111111111111100000000000000000000000000000000
all_view.groupby(['file_dir_id']).agg({'size': sum, 'file_dir': list}).sort_values('size').compute()

,size,file_dir
file_dir_id,,
0,0,"[, , , , , , , , , , , , , , , , , , , , , , ,..."
113104467636781056,0,"[, , , , , , , , , , , , , , , , , , , , , , ,..."
1571755608527339520,0,"[/p/gpfs1/iopp/temp/montage/2854964, /p/gpfs1/..."
5347462496607272960,0,"[/p/gpfs1/iopp/temp/montage, /p/gpfs1/iopp/tem..."
8459554455933681664,24072977,"[/p/gpfs1/iopp/temp/montage/2854964/30, /p/gpf..."
...,...,...
7668160539272413184,1726748997,"[/p/gpfs1/iopp/temp/montage/2854964/29/run, /p..."
7672382663923073024,3254544638,"[/p/gpfs1/iopp/temp/montage/2854964/0/run, /p/..."
7701937536477691904,4237098833,"[/p/gpfs1/iopp/temp/montage/2854964/25/run, /p..."


time: 26.5 s (started: 2023-06-21 15:30:23 -07:00)


In [87]:
all_view.index.count().compute()

6989131

time: 4.73 s (started: 2023-06-21 16:18:28 -07:00)


In [112]:
!mv /p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet/global.json ../

time: 697 ms (started: 2023-06-21 16:45:58 -07:00)


In [27]:
log_dir2 = "/p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet"
all_view2 = pd.read_parquet(f"{log_dir2}")
all_view2

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,...,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
0,1,111084,0,314672,0,3,0.242405,0.242414,__xstat64,0,...,mImgtbl,mImgtbl#lassen247#111084#35184372403504,/p/gpfs1/iopp/temp/montage/2854964/2,0,0,0.000000,0.000009,2424094,1571755611461267775,-4245390515581891276
1,2,111084,0,314672,0,3,0.242502,0.242732,fopen64,0,...,mImgtbl,mImgtbl#lassen247#111084#35184372403504,/p/gpfs1/iopp/temp/montage/2854964/2/run/Kimag...,0,0,0.000000,0.000230,2426168,7638324191939094055,-4245390515581891276
2,3,111084,0,314672,0,3,0.242851,0.242868,opendir,0,...,mImgtbl,mImgtbl#lassen247#111084#35184372403504,/p/gpfs1/iopp/temp/montage/2854964/2,0,0,0.000000,0.000017,2428593,1571755611461267775,-4245390515581891276
3,4,111084,0,314672,0,3,0.242946,0.242956,__xstat64,0,...,mImgtbl,mImgtbl#lassen247#111084#35184372403504,/p/gpfs1/iopp/temp/montage/2854964/2,0,0,0.000000,0.000010,2429507,1571755611461267775,-4245390515581891276
4,5,111084,0,314672,0,3,0.243006,0.243016,__xstat64,0,...,mImgtbl,mImgtbl#lassen247#111084#35184372403504,/p/gpfs1/iopp/temp/montage/2854964,0,0,0.000000,0.000010,2430107,5347462497940582382,-4245390515581891276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6989126,2332457,41731,31,317888,0,2,1.604730,1.604747,fwrite,0,...,mAddMPI,mAddMPI#lassen246#41731#35184372406720,/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc33...,96920,0,5469.237793,0.000017,16047384,7686174940287124680,8238577613900339652
6989127,2332458,41731,31,317888,0,2,1.604971,1.604992,fwrite,0,...,mAddMPI,mAddMPI#lassen246#41731#35184372406720,/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc33...,96920,0,4359.911133,0.000021,16049814,7686174937900816169,8238577613900339652
6989128,2332459,41731,31,317888,0,3,1.605062,1.605241,fclose,0,...,mAddMPI,mAddMPI#lassen246#41731#35184372406720,/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc33...,0,0,0.000000,0.000179,16051513,7686174940287124680,8238577613900339652
6989129,2332460,41731,31,317888,0,3,1.605244,1.605379,fclose,0,...,mAddMPI,mAddMPI#lassen246#41731#35184372406720,/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc33...,0,0,0.000000,0.000135,16053112,7686174937900816169,8238577613900339652


time: 19.4 s (started: 2023-07-03 19:57:28 -07:00)


### Data preparation

In [28]:
all_view_200k = all_view2.loc[:200_000]
all_view_1m = all_view2.loc[:1_000_000]
all_view_5m = all_view2.loc[:5_000_000]

time: 1.03 ms (started: 2023-07-03 19:57:48 -07:00)


### Iteration 

In [29]:
%%timeit -r 10 -n 1

size = 0
for row in all_view_200k.itertuples():
    size = size + getattr(row, 'size')
# size

1.15 s ± 14.8 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 11.5 s (started: 2023-07-03 19:57:48 -07:00)


In [30]:
%%timeit -r 10 -n 1

size = 0
for row in all_view_1m.itertuples():
    size = size + getattr(row, 'size')
# size

5.72 s ± 47.3 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 57.2 s (started: 2023-07-03 19:57:59 -07:00)


In [31]:
%%timeit -r 10 -n 1

size = 0
for row in all_view_5m.itertuples():
    size = size + getattr(row, 'size')
# size

29.1 s ± 1.12 s per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 4min 51s (started: 2023-07-03 19:58:56 -07:00)


In [32]:
!lscpu 

Architecture:          ppc64le
Byte Order:            Little Endian
CPU(s):                128
On-line CPU(s) list:   0-127
Thread(s) per core:    4
Core(s) per socket:    16
Socket(s):             2
NUMA node(s):          6
Model:                 2.1 (pvr 004e 1201)
Model name:            POWER9, altivec supported
CPU max MHz:           3800.0000
CPU min MHz:           2300.0000
L1d cache:             32K
L1i cache:             32K
L2 cache:              512K
L3 cache:              10240K
NUMA node0 CPU(s):     0-63
NUMA node8 CPU(s):     64-127
NUMA node252 CPU(s):   
NUMA node253 CPU(s):   
NUMA node254 CPU(s):   
NUMA node255 CPU(s):   
time: 278 ms (started: 2023-07-03 20:03:47 -07:00)


In [109]:
all_view_5m.index.count().compute()

6989131

time: 4.76 s (started: 2023-06-21 16:43:48 -07:00)


## Eval IV: Preprocessing - Query on raw vs. preprocessed data

In [33]:
def find_func(df):
    t0 = time()
    cond = (df['func_id'].str.contains('read')) | (df['func_id'].str.contains('write'))
    df[cond]
    t1 = time()
    df[~cond]
    t2 = time() - t1
    print(f"data,{t1-t0}")
    print(f"metadata,{t2}")


def find_func2(df):
    t0 = time()
    df[df['io_cat'].isin([1, 2])]
    t1 = time()
    df[df['io_cat'] == 3]
    t2 = time() - t1
    print(f"data,{t1-t0}")
    print(f"metadata,{t2}")

time: 2.38 ms (started: 2023-07-03 20:03:48 -07:00)


In [34]:
%%timeit -r 10 -n 1

find_func(all_view_200k)

data,0.35185909271240234
metadata,0.003263711929321289
data,0.33129191398620605
metadata,0.0036649703979492188
data,0.32831907272338867
metadata,0.003201007843017578
data,0.33491039276123047
metadata,0.003296375274658203
data,0.3273313045501709
metadata,0.0032303333282470703
data,0.3390803337097168
metadata,0.003599405288696289
data,0.34233856201171875
metadata,0.003646373748779297
data,0.3277597427368164
metadata,0.0032143592834472656
data,0.32671451568603516
metadata,0.003208160400390625
data,0.3285980224609375
metadata,0.003377676010131836
337 ms ± 7.93 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 3.38 s (started: 2023-07-03 20:03:48 -07:00)


In [35]:
%%timeit -r 10 -n 1

find_func2(all_view_200k)

data,0.022030115127563477
metadata,0.0036873817443847656
data,0.02002429962158203
metadata,0.0027441978454589844
data,0.01863884925842285
metadata,0.0034470558166503906
data,0.02010059356689453
metadata,0.0031795501708984375
data,0.02104496955871582
metadata,0.004355430603027344
data,0.01938605308532715
metadata,0.0030698776245117188
data,0.018854379653930664
metadata,0.0032660961151123047
data,0.018665313720703125
metadata,0.0031821727752685547
data,0.019803285598754883
metadata,0.0032045841217041016
data,0.019085168838500977
metadata,0.0037970542907714844
23.2 ms ± 1.29 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 235 ms (started: 2023-07-03 20:03:51 -07:00)


In [36]:
%%timeit -r 10 -n 1

find_func(all_view_1m)

data,1.7351508140563965
metadata,0.005721569061279297
data,1.6270883083343506
metadata,0.007005214691162109
data,1.6432061195373535
metadata,0.006081104278564453
data,1.6307802200317383
metadata,0.00590968132019043
data,1.6365070343017578
metadata,0.006067514419555664
data,1.6450905799865723
metadata,0.00578618049621582
data,1.6401088237762451
metadata,0.00562286376953125
data,1.6610126495361328
metadata,0.0061435699462890625
data,1.6280241012573242
metadata,0.006081104278564453
data,1.628856897354126
metadata,0.005987882614135742
1.65 s ± 30.8 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 16.5 s (started: 2023-07-03 20:03:51 -07:00)


In [37]:
%%timeit -r 10 -n 1

find_func2(all_view_1m)

data,0.10647368431091309
metadata,0.006810665130615234
data,0.10039424896240234
metadata,0.005963802337646484
data,0.10053801536560059
metadata,0.005600452423095703
data,0.10324645042419434
metadata,0.005883455276489258
data,0.10418081283569336
metadata,0.0061914920806884766
data,0.10621500015258789
metadata,0.005346059799194336
data,0.10210371017456055
metadata,0.0074040889739990234
data,0.10138583183288574
metadata,0.005728721618652344
data,0.10110235214233398
metadata,0.006037473678588867
data,0.1017918586730957
metadata,0.005674600601196289
109 ms ± 2.28 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 1.09 s (started: 2023-07-03 20:04:08 -07:00)


In [38]:
%%timeit -r 10 -n 1

find_func(all_view_5m)

data,8.539624452590942
metadata,0.0176241397857666
data,8.196680068969727
metadata,0.01795673370361328
data,8.175743341445923
metadata,0.018639326095581055
data,8.227416038513184
metadata,0.020246267318725586
data,8.241168975830078
metadata,0.021834850311279297
data,8.216010570526123
metadata,0.019608020782470703
data,8.247225761413574
metadata,0.01794600486755371
data,8.247453451156616
metadata,0.019954681396484375
data,8.19602346420288
metadata,0.01918196678161621
data,8.186300039291382
metadata,0.01942276954650879
8.27 s ± 100 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 1min 22s (started: 2023-07-03 20:04:09 -07:00)


In [39]:
%%timeit -r 10 -n 1

find_func2(all_view_5m)

data,0.5460391044616699
metadata,0.015449285507202148
data,0.5322365760803223
metadata,0.016766786575317383
data,0.534451961517334
metadata,0.016060829162597656
data,0.539341926574707
metadata,0.015818119049072266
data,0.5393798351287842
metadata,0.017310142517089844
data,0.54140305519104
metadata,0.016079187393188477
data,0.5332987308502197
metadata,0.01547861099243164
data,0.5393416881561279
metadata,0.016664743423461914
data,0.5380587577819824
metadata,0.017776012420654297
data,0.5386464595794678
metadata,0.01598978042602539
555 ms ± 3.84 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 5.55 s (started: 2023-07-03 20:05:32 -07:00)


In [171]:
def calc_hlm_view(ddf, persist=False):
    t0 = time()
    hlm_view = ddf[(ddf['cat'] == 0) & (ddf['io_cat'].isin([1, 2, 3]))] \
        .groupby(['file_id']) \
        .agg({
            'index': 'count',
            'duration': sum,
            'size': sum,
        }) \
        .reset_index()
    t1 = time()
    if persist:
        hlm_view = hlm_view.persist()
        # wait(hlm_view)
    t2 = time()
    hlm_view['bw'] = hlm_view['size'] / hlm_view['duration']
    hlm_view['bw_norm'] = hlm_view['bw'] / hlm_view['bw'].sum()
    hlm_view['intensity'] = hlm_view['bw'] / hlm_view['index']
    hlm_view['dur_per'] = hlm_view['duration'] / hlm_view['duration'].max()
    final_view = hlm_view.compute()
    t3 = time()
    print(f"q1,{t1-t0}")
    print(f"persist,{t2-t1}")
    print(f"q2,{t3-t2}")

time: 2.31 ms (started: 2023-06-21 17:20:27 -07:00)


In [142]:
all_view_200k_ddf = dd.from_pandas(all_view_200k, npartitions=20)
all_view_1m_ddf = dd.from_pandas(all_view_1m, npartitions=100)
all_view_5m_ddf = dd.from_pandas(all_view_5m, npartitions=500)

time: 9.88 s (started: 2023-06-21 17:08:22 -07:00)


In [144]:
all_view_25m_ddf = dd.from_pandas(dd.concat([all_view_5m_ddf]*5).compute(), npartitions=2500)

time: 1min 32s (started: 2023-06-21 17:09:50 -07:00)


In [172]:
calc_hlm_view(all_view_200k_ddf)

q1,0.037634849548339844
persist,2.384185791015625e-07
q2,0.3123667240142822
time: 351 ms (started: 2023-06-21 17:20:31 -07:00)


In [173]:
calc_hlm_view(all_view_200k_ddf, True)

q1,0.02421855926513672
persist,0.014767646789550781
q2,0.31082749366760254
time: 351 ms (started: 2023-06-21 17:20:34 -07:00)


In [174]:
calc_hlm_view(all_view_1m_ddf)

q1,0.0230865478515625
persist,4.76837158203125e-07
q2,0.886458158493042
time: 911 ms (started: 2023-06-21 17:20:35 -07:00)


In [175]:
calc_hlm_view(all_view_1m_ddf, True)

q1,0.02448439598083496
persist,0.03750467300415039
q2,1.170438528060913
time: 1.23 s (started: 2023-06-21 17:20:37 -07:00)


In [176]:
calc_hlm_view(all_view_5m_ddf)

q1,0.025345563888549805
persist,4.76837158203125e-07
q2,3.9722607135772705
time: 4 s (started: 2023-06-21 17:20:40 -07:00)


In [177]:
calc_hlm_view(all_view_5m_ddf, True)

q1,0.024511098861694336
persist,0.14873647689819336
q2,3.4780001640319824
time: 3.65 s (started: 2023-06-21 17:20:44 -07:00)


In [178]:
calc_hlm_view(all_view_25m_ddf)

q1,0.06383562088012695
persist,4.76837158203125e-07
q2,23.824289083480835
time: 23.9 s (started: 2023-06-21 17:20:47 -07:00)


In [179]:
calc_hlm_view(all_view_25m_ddf, True)

q1,1.176560878753662
persist,0.6906352043151855
q2,23.874505758285522
time: 25.7 s (started: 2023-06-21 17:21:11 -07:00)
